In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import shuffle
from keras.utils import to_categorical
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from sklearn.preprocessing import scale
import tensorflow as tf
from keras.layers import Dense
import pickle
from sklearn.metrics import accuracy_score

Read data from excel files

In [2]:
train=pd.read_excel("train.xlsx")
puppies=pd.read_excel("puppies.xlsx")

jOIN THE DATA.

In [3]:
df=train.join(puppies.set_index('PuppyNumber'),on="PuppyNumber",how='left',lsuffix='', rsuffix='_other')
df.drop(columns=["PuppyNumber"],inplace=True)
df.rename(columns=lambda x: x.strip(), inplace=True)
df['breed']=df['breed'].apply(lambda x: x.strip())

DROP COLUMNS WHICH ARE  LESS SIGNIFICANT

In [4]:
df.drop(columns=['Adult weight',
       'Number of litters', 'Average number of puppies per litter',
       'Total number of puppies','Parent Age','breed','sex'],inplace=True)

In [5]:

# categories = [('sex',['M','F']),('breed',['German Shepherd', 'English Mastiff', 'Maltese'])]
# ohe_columns = [x[0] for x in categories]
# ohe_categories = [x[1] for x in categories]
# encoding= OneHotEncoder(categories=ohe_categories)
# transformer = make_column_transformer((encoding, ohe_columns), remainder='passthrough')

# # We convert the resulting arrays to DataFrames
# dft=transformer.fit_transform(df)
# df=pd.DataFrame(dft,columns=transformer.get_feature_names_out(),index=df.index)
# df.rename(columns={"remainder__Health":"Health"},inplace=True)

Specifying X and Y dataset

In [6]:
X=df.drop(columns=['Health'],axis=1)
X=np.array(X)
df['Health']=df['Health'].astype('category')
Y=np.array(df['Health'].cat.codes)

Split data into test and train

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.1, random_state = 1,shuffle=True)
x_train= np.asarray(x_train).astype('float32')
x_test= np.asarray(x_test).astype('float32')
# x_train.shape, y_train.shape, x_test.shape, y_test.shape

Model Training

In [8]:
best_dct_model=None
best=0

1. Decision Tree Classifier

In [9]:
for i in range (1,20):
    dct_model=DecisionTreeClassifier(splitter='random',criterion="entropy")
    dct_model.fit(x_train,y_train)
    dct_y_pred=dct_model.predict(x_test)
    score =accuracy_score(y_test,dct_y_pred)
    if score>best:
        best_dct_model=dct_model
        best=score

best

0.7160493827160493

2. Random Forest

In [10]:
def try_rf(n):
    random_forest_model=RandomForestClassifier(n_estimators=n,random_state=42)
    random_forest_model.fit(x_train,y_train)
    random_forest_y_pred=random_forest_model.predict(x_test)
    score =accuracy_score(y_test,random_forest_y_pred)
    return random_forest_model,random_forest_y_pred,score

best_score = n = 0
random_forest_best_model = knn_y_pred = None
for i in range(200, 300,20):
    random_forest_model, y_pred, score = try_rf(i)
    if score > best_score:
        best_score = score
        random_forest_best_model = random_forest_model
        random_forest_y_pred = y_pred
        n = i
print("Best Score", best_score, n)

Best Score 0.7160493827160493 280


3.  Logistic Regression

In [11]:
lg_model=LogisticRegression(max_iter=10000)
lg_model.fit(x_train,y_train)
lg_y_pred=lg_model.predict(x_test)
score =accuracy_score(y_test,lg_y_pred)
score

0.345679012345679

4. Support  Vector Machine

In [12]:
svc_model=SVC(max_iter=30000,probability=True)
svc_model.fit(x_train,y_train)
svc_pred=svc_model.predict(x_test)
score =accuracy_score(y_test,svc_pred)
score

0.3395061728395062

5. Naive Bayes

In [13]:
nb_model=GaussianNB()
nb_model.fit(x_train,y_train)
nb_y_pred=nb_model.predict(x_test)
score =accuracy_score(y_test,nb_y_pred)
score

0.35185185185185186

6. Kth Nearest Neighbour

In [14]:
def try_knn(n_neighbors):
    knn_model = KNeighborsClassifier(n_neighbors=n_neighbors,p=5,n_jobs=20,algorithm="kd_tree")
    knn_model.fit(x_train, y_train)
    knn_y_pred = knn_model.predict(x_test)
    score = accuracy_score(y_test, knn_y_pred)
    return knn_model, knn_y_pred, score


best_score = n = 0
knn_best_model = knn_y_pred = None
for i in range(1, 100):
    knn_model, y_pred, score = try_knn(i)
    if score > best_score:
        best_score = score
        knn_best_model = knn_model
        knn_y_pred = y_pred
        n = i
print("Best Score", best_score, n)

Best Score 0.4444444444444444 96


7. Neural Network

In [ ]:
in_dim = len(df.columns)-1
model = Sequential()
model.add(Dense(64, input_dim = in_dim, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.fit(x_train, y_train, epochs = 5, batch_size = 3,verbose=False)
scores = model.evaluate(x_test, y_test)

for i, m in enumerate(model.metrics_names):
    print("\n%s: %.3f"% (m, scores[i]))

Save the best Models

In [16]:
with open('puppy_management_system_random_forest_model.pkl', 'wb') as file:
    pickle.dump(random_forest_best_model, file)
with open('puppy_management_system_decision_tree_model.pkl', 'wb') as file:
    pickle.dump(best_dct_model, file)

In [17]:
  # Load the saved model
with open('puppy_management_system_random_forest_model.pkl', 'rb') as file:
    loaded_rf_classifier = pickle.load(file)
# Load the saved model
with open('puppy_management_system_decision_tree_model.pkl', 'rb') as file:
    loaded_dct_classifier = pickle.load(file)

rf_y_pred=loaded_rf_classifier.predict(x_test)
rf_score =accuracy_score(y_test,rf_y_pred)

dct_y_pred=loaded_dct_classifier.predict(x_test)
dct_score =accuracy_score(y_test,dct_y_pred)

print(f"RF: {rf_score} DCT: {dct_score}")

RF: 0.7160493827160493 DCT: 0.7160493827160493
